In [0]:
%sql
CREATE OR REPLACE VIEW transacoes_db.feature_store.in_live_features AS
WITH tx_com_long AS (
  SELECT
    *,
    CAST(data AS LONG) AS data_em_segundos
  FROM
    transacoes_db.gold.transacoes_balanced_model
),
tx_features_realtime AS (
  SELECT
    tx.*,
    -- Pagador: últimas 24h
    COUNT(1) OVER (
      PARTITION BY id_conta_origem
      ORDER BY data_em_segundos
      RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING
    ) AS pagador_txs_ultimas_24h,
    COALESCE(
      SUM(valor) OVER (
        PARTITION BY id_conta_origem
        ORDER BY data_em_segundos
        RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING
      ), 0
    ) AS pagador_valor_ultimas_24h,
    -- Recebedor: última 1h
    COUNT(1) OVER (
      PARTITION BY id_conta_destino
      ORDER BY data_em_segundos
      RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING
    ) AS recebedor_txs_ultima_1h,
    COALESCE(
      SUM(valor) OVER (
        PARTITION BY id_conta_destino
        ORDER BY data_em_segundos
        RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING
      ), 0
    ) AS recebedor_valor_ultima_1h,
    -- Lag do pagador
    data_em_segundos - LAG(data_em_segundos, 1, 0) OVER (
      PARTITION BY id_conta_origem
      ORDER BY data_em_segundos
    ) AS pagador_segundos_desde_ultima_tx
  FROM
    tx_com_long tx
)
SELECT
  -- Colunas da transação
  ft.valor AS valor_transacao,
  ft.data AS data_transacao,
  ft.id_conta_origem AS id_conta_pagador,
  ft.id_conta_destino AS id_conta_recebedor,
  ft.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  ft.id_finalidade_pix AS finalidade_pix_id,
  ft.is_fraud AS transacao_fraudulenta,
  ft.fraud_type AS tipo_fraude,

  -- Pagador: Perfil
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,

  -- Recebedor: Perfil
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  -- Features de tempo real
  ft.pagador_txs_ultimas_24h,
  ft.pagador_valor_ultimas_24h,
  ft.recebedor_txs_ultima_1h,
  ft.recebedor_valor_ultima_1h,
  ft.pagador_segundos_desde_ultima_tx
FROM
  tx_features_realtime AS ft
  LEFT JOIN transacoes_db.copper.contas AS conta_orig
    ON ft.id_conta_origem = conta_orig.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_orig
    ON conta_orig.id_cliente = cliente_orig.id
  LEFT JOIN transacoes_db.copper.contas AS conta_dest
    ON ft.id_conta_destino = conta_dest.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_dest
    ON conta_dest.id_cliente = cliente_dest.id

In [0]:
%sql
CREATE OR REPLACE VIEW transacoes_db.feature_store.historical_data_features AS 
-- Bloco 1: CTE para pré-agregar as chaves PIX
-- Isso previne a duplicação de linhas na junção principal e melhora a performance.
WITH contas_enriquecidas AS (
  SELECT
    c.id,
    c.id_cliente,
    c.saldo,
    c.aberta_em,
    c.id_tipo_conta,
    c.ispb_instituicao,
    c.estado_ibge,
    c.municipio_ibge,
    -- Nós NÃO usamos 'is_high_risk' para evitar vazamento de dados
    
    -- Agregações da tabela de chaves
    coalesce(k.qtd_chaves, 0) AS qtd_chaves,
    k.primeira_chave_em,
    k.ultima_chave_em
  FROM
    transacoes_db.copper.contas c
  LEFT JOIN (
    -- Subquery que calcula as métricas por conta
    SELECT
      id_conta,
      COUNT(id) AS qtd_chaves,
      MIN(cadastrada_em) AS primeira_chave_em,
      MAX(cadastrada_em) AS ultima_chave_em
    FROM
      transacoes_db.copper.chaves_pix
    GROUP BY
      id_conta
  ) k ON c.id = k.id_conta
)

-- Bloco 2: Query Principal (Engenharia de Features)
SELECT
  -- ===================================================================
  -- 1. TARGETS (As respostas que a IA deve aprender)
  -- ===================================================================
  tx.is_fraud AS transacao_fraudulenta,
  coalesce(tx.fraud_type, 'Legitima') AS tipo_fraude,
  
  -- ===================================================================
  -- 2. FEATURES BASE (Dados brutos da transação)
  -- ===================================================================
  tx.valor AS valor_transacao,
  tx.data AS data_transacao,
  coalesce(tx.id_tipo_iniciacao_pix, -1) AS tipo_iniciacao_pix_id,
  coalesce(tx.id_finalidade_pix, -1) AS finalidade_pix_id,

  -- ===================================================================
  -- 3. FEATURES DE VELOCIDADE (Quão rápido as coisas acontecem?)
  -- ===================================================================
  -- Mitigação: Nenhuma necessária. Esta lógica já retorna 0 em vez de NULL.
  hour(tx.data) AS hora_do_dia,
  
  (unix_timestamp(tx.data) - unix_timestamp(
      LAG(tx.data, 1, tx.data) OVER (
        PARTITION BY tx.id_conta_origem 
        ORDER BY tx.data ASC
      )
  )) AS pagador_segundos_desde_ult_tx,
  
  (unix_timestamp(tx.data) - unix_timestamp(
      LAG(tx.data, 1, tx.data) OVER (
        PARTITION BY tx.id_conta_destino 
        ORDER BY tx.data ASC
      )
  )) AS recebedor_segundos_desde_ult_tx,

  -- ===================================================================
  -- 4. FEATURES DE CONTEXTO DA CONTA (Sinais de "Laranja")
  -- ===================================================================
  -- Mitigação: coalesce(..., 0) para contas que não forem encontradas no JOIN
  coalesce(DATEDIFF(date(tx.data), date(conta_orig.aberta_em)), 0) AS pagador_dias_de_conta,
  coalesce(DATEDIFF(date(tx.data), date(conta_dest.aberta_em)), 0) AS recebedor_dias_de_conta,

  -- Mitigação: coalesce(..., 0) + 0.01 para evitar NULLs e divisão por zero
  (
    tx.valor / (coalesce(conta_orig.saldo, 0) + 0.01)
  ) AS tx_vs_saldo_pagador_ratio,
  
  -- Mitigação: coalesce(..., -1) para idade desconhecida (ex: cliente não encontrado)
  coalesce(
    floor(DATEDIFF(date(tx.data), date(cliente_orig.nascido_em)) / 365.25),
    -1
  ) AS pagador_idade_cliente_anos,
  
  coalesce(
    floor(DATEDIFF(date(tx.data), date(cliente_dest.nascido_em)) / 365.25),
    -1
  ) AS recebedor_idade_cliente_anos,

  -- ===================================================================
  -- 5. FEATURES DE CHAVE PIX (Simplificadas pela CTE)
  -- ===================================================================
  -- Mitigação: A CTE já usa coalesce(..., 0)
  conta_dest.qtd_chaves AS recebedor_qtd_chaves_pix,

  -- Mitigação: coalesce(..., 9999) para "tempo muito longo / nunca cadastrou"
  coalesce(
    DATEDIFF(date(tx.data), date(conta_dest.ultima_chave_em)),
    9999
  ) AS recebedor_dias_desde_ult_chave_pix,
  
  -- "Sinal de Laranja": Dias entre abertura da conta e 1ª chave
  -- Mitigação: coalesce(..., 9999) para "tempo muito longo / nunca cadastrou"
  coalesce(
    DATEDIFF(date(conta_dest.primeira_chave_em), date(conta_dest.aberta_em)),
    9999
  ) AS recebedor_dias_abertura_ate_1a_chave,

  -- ===================================================================
  -- 6. FEATURES DE TRIANGULAÇÃO (Diversidade de Contas)
  -- ===================================================================
  -- Mitigação: Nenhuma necessária. Esta lógica é robusta.
  size(array_distinct(collect_list(tx.id_conta_origem) OVER (
      PARTITION BY tx.id_conta_destino, date(tx.data)
  ))) AS recebedor_pagadores_unicos_dia,

  size(array_distinct(collect_list(tx.id_conta_destino) OVER (
      PARTITION BY tx.id_conta_origem, date(tx.data)
  ))) AS pagador_recebedores_unicos_dia,

  -- ===================================================================
  -- 7. FEATURES DE CONTAGEM E HISTÓRICO
  -- ===================================================================
  -- Mitigação: Nenhuma necessária. Esta lógica é robusta.
  (ROW_NUMBER() OVER (
      PARTITION BY tx.id_conta_origem, tx.id_conta_destino
      ORDER BY tx.data ASC 
    ) - 1) as qtd_transacoes_hist_pagador_recebedor, 

  COUNT(1) OVER (
      PARTITION BY tx.id_conta_origem, date(tx.data)
  ) as qtd_transacoes_dia_pagador,
  
  COUNT(1) OVER (
      PARTITION BY tx.id_conta_destino, date(tx.data)
  ) AS recebedor_total_txs_no_dia,

  -- ===================================================================
  -- 8. FEATURES ESTATÍSTICAS (Detecção de Anomalia)
  -- ===================================================================
  -- Mitigação: coalesce(..., 0) para a 1ª transação (histórico nulo)
  coalesce(
    AVG(tx.valor) OVER (
        PARTITION BY tx.id_conta_origem ORDER BY tx.data ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ), 0
  ) AS pagador_media_valor_hist,

  -- Mitigação: coalesce(..., 0) para a 1ª/2ª transação (histórico nulo)
  coalesce(
    STDDEV(tx.valor) OVER (
        PARTITION BY tx.id_conta_origem ORDER BY tx.data ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ), 0
  ) AS pagador_stddev_valor_hist,

  -- ===================================================================
  -- 9. FEATURES CATEGÓRICAS (IDs brutos)
  -- ===================================================================
  -- Mitigação: coalesce(..., -1) para IDs desconhecidos
  coalesce(conta_orig.id_tipo_conta, -1) AS pagador_tipo_conta_id,
  coalesce(cliente_orig.id_natureza, -1) AS pagador_natureza_id,
  coalesce(conta_dest.id_tipo_conta, -1) AS recebedor_tipo_conta_id,
  coalesce(cliente_dest.id_natureza, -1) AS recebedor_natureza_id

FROM
  transacoes_db.copper.transacoes AS tx

-- Joins do Pagador (Usando a CTE)
 LEFT JOIN contas_enriquecidas AS conta_orig
  ON tx.id_conta_origem = conta_orig.id
 LEFT JOIN transacoes_db.copper.clientes AS cliente_orig
  ON conta_orig.id_cliente = cliente_orig.id

-- Joins do Recebedor (Usando a CTE)
 LEFT JOIN contas_enriquecidas AS conta_dest
  ON tx.id_conta_destino = conta_dest.id
 LEFT JOIN transacoes_db.copper.clientes AS cliente_dest
  ON conta_dest.id_cliente = cliente_dest.id
  
-- Outros Joins Dimensionais (Opcional)
 LEFT JOIN transacoes_db.copper.finalidade_pix AS finalidade_pix
  ON tx.id_finalidade_pix = finalidade_pix.id

ORDER BY
  tx.data ASC